# Донавчання детектора об'єктів на датасеті ArTaxOr

Цей notebook містить повний пайплайн для донавчання моделі YOLOv8 на датасеті [ArTaxOr (Arthropod Taxonomy Orders Object Detection Dataset)](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset) з Kaggle.


## 1. Встановлення залежностей


In [ ]:
%pip install ultralytics>=8.0.0 -q
%pip install opencv-python>=4.8.0 -q
%pip install pillow>=10.0.0 -q
%pip install pyyaml>=6.0 -q
%pip install tqdm>=4.65.0 -q


## 2. Імпорт бібліотек


In [ ]:
import os
import json
import shutil
import random
from pathlib import Path
from PIL import Image
import xml.etree.ElementTree as ET
from tqdm import tqdm
from ultralytics import YOLO
import time


## 3. Завантаження датасету з Kaggle


In [ ]:
# Налаштування шляхів
INPUT_DIR = "/kaggle/input/arthropod-taxonomy-orders-object-detection-dataset"
OUTPUT_DIR = "/kaggle/working"
YOLO_DATASET_DIR = os.path.join(OUTPUT_DIR, "yolo_dataset")

print(f"Вхідна директорія: {INPUT_DIR}")
print(f"Робоча директорія: {OUTPUT_DIR}")

# Автоматичний пошук датасету
DATASET_DIR = None

# Перевіряємо різні можливі шляхи
possible_paths = [
    os.path.join(INPUT_DIR, "ArTaxOr"),
    INPUT_DIR,  # Датасет може бути безпосередньо в INPUT_DIR
]

# Також шукаємо рекурсивно в INPUT_DIR
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        # Шукаємо директорії, які містять XML або JSON файли (анотації)
        xml_files = [f for f in files if f.endswith('.xml')]
        json_files = [f for f in files if f.endswith('.json')]
        if xml_files or json_files:
            DATASET_DIR = root
            break
        # Або шукаємо директорію з назвою ArTaxOr
        if 'ArTaxOr' in dirs:
            DATASET_DIR = os.path.join(root, 'ArTaxOr')
            break

# Якщо не знайшли, перевіряємо стандартні шляхи
if DATASET_DIR is None:
    for path in possible_paths:
        if os.path.exists(path):
            # Перевіряємо, чи є там XML або JSON файли
            for root, dirs, files in os.walk(path):
                xml_files = [f for f in files if f.endswith('.xml')]
                json_files = [f for f in files if f.endswith('.json')]
                if xml_files or json_files:
                    DATASET_DIR = path
                    break
            if DATASET_DIR:
                break

if DATASET_DIR and os.path.exists(DATASET_DIR):
    print(f"\nДатасет знайдено в: {DATASET_DIR}")
    # Показуємо структуру датасету (перші 3 рівні)
    print("\nСтруктура датасету:")
    for root, dirs, files in os.walk(DATASET_DIR):
        level = root.replace(DATASET_DIR, '').count(os.sep)
        if level > 2:  # Обмежуємо глибину для читабельності
            continue
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        # Показуємо файли анотацій
        annotation_files = [f for f in files if f.endswith(('.xml', '.json'))]
        if annotation_files:
            for file in annotation_files[:3]:
                print(f"{subindent}{file}")
            if len(annotation_files) > 3:
                print(f"{subindent}... та ще {len(annotation_files) - 3} файлів анотацій")
        # Показуємо зображення
        image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        if image_files and not annotation_files:
            for file in image_files[:3]:
                print(f"{subindent}{file}")
            if len(image_files) > 3:
                print(f"{subindent}... та ще {len(image_files) - 3} зображень")
else:
    print(f"\nПОПЕРЕДЖЕННЯ: Датасет не знайдено")
    print(f"Перевірено шляхи:")
    for path in possible_paths:
        exists = "існує" if os.path.exists(path) else "не існує"
        print(f"  - {path} ({exists})")
    print("\nПереконайтеся, що:")
    print("1. Датасет додано до notebook через Add Data -> Search Datasets")
    print("2. Назва датасету: arthropod-taxonomy-orders-object-detection-dataset")
    print("3. Автор датасету: mistag")


## 4. Функції для конвертації датасету у формат YOLO


In [ ]:
def parse_pascal_voc(xml_path):
    """
    Парсить XML файл у форматі Pascal VOC
    Повертає список bbox у форматі [class_id, x_center, y_center, width, height] (нормалізовані)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Отримуємо розміри зображення
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    boxes = []
    for obj in root.findall('object'):
        # Отримуємо клас (може бути name або class)
        class_name = obj.find('name').text
        
        # Отримуємо bbox
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        # Конвертуємо в YOLO формат (нормалізовані координати центру та розміри)
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        boxes.append({
            'class': class_name,
            'bbox': [x_center, y_center, width, height]
        })
    
    return boxes, img_width, img_height

def parse_coco(json_path):
    """
    Парсить JSON файл у форматі COCO
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Створюємо мапи для швидкого пошуку
    images = {img['id']: img for img in data['images']}
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Групуємо анотації по зображенням
    annotations_by_image = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(ann)
    
    return images, categories, annotations_by_image

def convert_coco_to_yolo(images, categories, annotations_by_image, class_mapping):
    """
    Конвертує анотації COCO у формат YOLO
    """
    result = {}
    for image_id, image_info in images.items():
        img_width = image_info['width']
        img_height = image_info['height']
        file_name = image_info['file_name']
        
        boxes = []
        if image_id in annotations_by_image:
            for ann in annotations_by_image[image_id]:
                category_id = ann['category_id']
                class_name = categories[category_id]
                
                if class_name not in class_mapping:
                    continue
                
                class_id = class_mapping[class_name]
                
                # COCO bbox формат: [x_min, y_min, width, height]
                x_min, y_min, width, height = ann['bbox']
                
                # Конвертуємо в YOLO формат
                x_center = (x_min + width / 2) / img_width
                y_center = (y_min + height / 2) / img_height
                norm_width = width / img_width
                norm_height = height / img_height
                
                boxes.append({
                    'class_id': class_id,
                    'bbox': [x_center, y_center, norm_width, norm_height]
                })
        
        result[file_name] = boxes
    
    return result

def detect_format(dataset_path):
    """
    Визначає формат анотацій датасету
    """
    dataset_path = Path(dataset_path)
    
    # Перевірка на Pascal VOC (XML файли)
    xml_files = list(dataset_path.rglob("*.xml"))
    if xml_files:
        return "pascal_voc", xml_files
    
    # Перевірка на COCO (JSON файли)
    json_files = list(dataset_path.rglob("*.json"))
    if json_files:
        # Шукаємо файл з annotations
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                    if 'images' in data and 'annotations' in data and 'categories' in data:
                        return "coco", json_file
            except:
                continue
    
    return None, None


In [ ]:
def convert_dataset(input_path, output_path, train_split=0.8, val_split=0.1):
    """
    Конвертує датасет у формат YOLO
    
    Args:
        input_path: Шлях до вхідного датасету
        output_path: Шлях для вихідного датасету YOLO
        train_split: Частка даних для навчання
        val_split: Частка даних для валідації
    """
    input_path = Path(input_path)
    output_path = Path(output_path)
    
    # Створюємо структуру директорій YOLO
    for split in ['train', 'val', 'test']:
        (output_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (output_path / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Визначаємо формат датасету
    format_type, format_data = detect_format(input_path)
    
    if format_type is None:
        print("Помилка: не вдалося визначити формат анотацій")
        print("Підтримувані формати: Pascal VOC (XML), COCO (JSON)")
        return
    
    print(f"Виявлено формат: {format_type}")
    
    # Збираємо всі класи
    class_names = set()
    
    if format_type == "pascal_voc":
        xml_files = format_data
        for xml_file in tqdm(xml_files, desc="Збір класів"):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                class_names.add(class_name)
        
        # Створюємо мапінг класів
        class_names = sorted(list(class_names))
        class_mapping = {name: idx for idx, name in enumerate(class_names)}
        
        # Конвертуємо файли
        all_files = []
        for xml_file in tqdm(xml_files, desc="Конвертація"):
            # Шукаємо відповідне зображення
            img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            img_file = None
            for ext in img_extensions:
                potential_img = xml_file.parent / (xml_file.stem + ext)
                if potential_img.exists():
                    img_file = potential_img
                    break
            
            if img_file is None:
                continue
            
            boxes, img_width, img_height = parse_pascal_voc(xml_file)
            
            # Записуємо YOLO формат
            yolo_labels = []
            for box in boxes:
                class_id = class_mapping[box['class']]
                bbox = box['bbox']
                yolo_labels.append(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
            
            all_files.append((img_file, yolo_labels))
    
    elif format_type == "coco":
        json_file = format_data
        images, categories, annotations_by_image = parse_coco(json_file)
        
        # Створюємо мапінг класів
        class_names = sorted(list(categories.values()))
        class_mapping = {name: idx for idx, name in enumerate(class_names)}
        
        # Конвертуємо
        yolo_data = convert_coco_to_yolo(images, categories, annotations_by_image, class_mapping)
        
        # Знаходимо зображення
        img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        all_files = []
        for file_name, boxes in tqdm(yolo_data.items(), desc="Конвертація"):
            img_file = None
            for ext in img_extensions:
                potential_img = input_path / file_name
                if potential_img.exists():
                    img_file = potential_img
                    break
                # Шукаємо рекурсивно
                for found_img in input_path.rglob(file_name):
                    img_file = found_img
                    break
            
            if img_file is None or not img_file.exists():
                continue
            
            yolo_labels = []
            for box in boxes:
                yolo_labels.append(
                    f"{box['class_id']} {box['bbox'][0]:.6f} {box['bbox'][1]:.6f} "
                    f"{box['bbox'][2]:.6f} {box['bbox'][3]:.6f}"
                )
            
            all_files.append((img_file, yolo_labels))
    
    # Розділяємо на train/val/test
    random.seed(42)
    random.shuffle(all_files)
    
    n_total = len(all_files)
    n_train = int(n_total * train_split)
    n_val = int(n_total * val_split)
    
    splits = {
        'train': all_files[:n_train],
        'val': all_files[n_train:n_train + n_val],
        'test': all_files[n_train + n_val:]
    }
    
    # Копіюємо файли та створюємо labels
    for split_name, files in splits.items():
        print(f"\nОбробка {split_name}: {len(files)} файлів")
        for img_file, yolo_labels in tqdm(files, desc=f"Копіювання {split_name}"):
            # Копіюємо зображення
            dest_img = output_path / split_name / 'images' / img_file.name
            shutil.copy2(img_file, dest_img)
            
            # Створюємо label файл
            label_file = output_path / split_name / 'labels' / (img_file.stem + '.txt')
            with open(label_file, 'w') as f:
                f.write('\n'.join(yolo_labels))
    
    # Створюємо файл з назвами класів
    classes_file = output_path / 'classes.txt'
    with open(classes_file, 'w') as f:
        f.write('\n'.join(class_names))
    
    # Створюємо data.yaml для YOLO
    yaml_content = f"""# ArTaxOr Dataset Configuration
path: {output_path.absolute()}
train: train/images
val: val/images
test: test/images

# Classes
nc: {len(class_names)}
names: {class_names}
"""
    
    yaml_file = output_path / 'data.yaml'
    with open(yaml_file, 'w') as f:
        f.write(yaml_content)
    
    print(f"\nКонвертацію завершено!")
    print(f"Вихідний датасет: {output_path}")
    print(f"Кількість класів: {len(class_names)}")
    print(f"Класи: {', '.join(class_names)}")
    print(f"Конфігурація: {yaml_file}")


In [ ]:
# Конвертуємо датасет
convert_dataset(
    input_path=DATASET_DIR,
    output_path=YOLO_DATASET_DIR,
    train_split=0.8,
    val_split=0.1
)


## 6. Перевірка структури конвертованого датасету


In [ ]:
# Перевіряємо структуру датасету
data_yaml = os.path.join(YOLO_DATASET_DIR, "data.yaml")
if os.path.exists(data_yaml):
    print(f"Файл конфігурації створено: {data_yaml}")
    with open(data_yaml, 'r') as f:
        print("\nВміст data.yaml:")
        print(f.read())
    
    # Показуємо статистику
    for split in ['train', 'val', 'test']:
        images_dir = os.path.join(YOLO_DATASET_DIR, split, 'images')
        labels_dir = os.path.join(YOLO_DATASET_DIR, split, 'labels')
        if os.path.exists(images_dir):
            num_images = len([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
            num_labels = len([f for f in os.listdir(labels_dir) if f.endswith('.txt')])
            print(f"\n{split.upper()}: {num_images} зображень, {num_labels} labels")
else:
    print(f"ПОМИЛКА: Файл конфігурації не знайдено: {data_yaml}")


## 7. Навчання моделі YOLOv8


In [ ]:
# Параметри навчання
MODEL_SIZE = 'n'  # n, s, m, l, x (nano, small, medium, large, xlarge)
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16
DEVICE = 'cuda' if os.environ.get('CUDA_VISIBLE_DEVICES') else 'cpu'
PROJECT_NAME = 'artaxor_training'

print(f"Параметри навчання:")
print(f"   - Модель: YOLOv8{MODEL_SIZE}")
print(f"   - Епохи: {EPOCHS}")
print(f"   - Розмір зображення: {IMG_SIZE}")
print(f"   - Батч: {BATCH_SIZE}")
print(f"   - Пристрій: {DEVICE}")
print(f"   - Датасет: {data_yaml}")


In [ ]:
# Завантажуємо модель
model_name = f'yolov8{MODEL_SIZE}.pt'
print(f"Завантаження попередньо навченої моделі: {model_name}")
model = YOLO(model_name)

print("Модель завантажено успішно!")


In [ ]:
# Початок навчання
print("\n" + "="*60)
print("ПОЧАТОК НАВЧАННЯ")
print("="*60)

start_time = time.time()

results = model.train(
    data=data_yaml,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    project=OUTPUT_DIR,
    name=PROJECT_NAME,
    save=True,
    save_period=10,  # Зберігати чекпоінт кожні 10 епох
    val=True,  # Валідація під час навчання
    plots=True,  # Генерувати графіки
    verbose=True,  # Показувати детальний прогрес навчання
    seed=42,  # Для відтворюваності
    deterministic=True,
    amp=True,  # Automatic Mixed Precision (швидше навчання)
    cos_lr=True,  # Cosine learning rate schedule
    close_mosaic=10,  # Вимкнути mosaic за 10 епох до кінця
)

end_time = time.time()
training_time = end_time - start_time
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("\n" + "="*60)
print("НАВЧАННЯ ЗАВЕРШЕНО!")
print("="*60)
print(f"Час навчання: {hours:02d}:{minutes:02d}:{seconds:02d}")
print("="*60)


## 8. Результати навчання


In [ ]:
# Шляхи до результатів
results_dir = os.path.join(OUTPUT_DIR, PROJECT_NAME)
best_model = os.path.join(results_dir, 'weights', 'best.pt')
last_model = os.path.join(results_dir, 'weights', 'last.pt')

print(f"Результати збережено в: {results_dir}")
print(f"Найкраща модель: {best_model}")
print(f"Остання модель: {last_model}")

# Виводимо метрики
if hasattr(results, 'results_dict'):
    print("\nМетрики навчання:")
    for key, value in results.results_dict.items():
        print(f"   {key}: {value}")

# Показуємо графіки результатів
results_png = os.path.join(results_dir, 'results.png')
if os.path.exists(results_png):
    print(f"\nГрафіки результатів: {results_png}")
    from IPython.display import Image, display
    display(Image(results_png))


## 9. Валідація моделі (опціонально)


In [ ]:
# Валідація найкращої моделі
if os.path.exists(best_model):
    print(f"Валідація моделі: {best_model}")
    validation_model = YOLO(best_model)
    
    val_results = validation_model.val(
        data=data_yaml,
        imgsz=IMG_SIZE,
        device=DEVICE
    )
    
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТИ ВАЛІДАЦІЇ:")
    print("="*60)
    print(f"   mAP50: {val_results.box.map50:.4f}")
    print(f"   mAP50-95: {val_results.box.map:.4f}")
    if hasattr(val_results.box, 'mp'):
        print(f"   Precision: {val_results.box.mp:.4f}")
    if hasattr(val_results.box, 'mr'):
        print(f"   Recall: {val_results.box.mr:.4f}")
    print("="*60)
else:
    print(f"ПОМИЛКА: Модель не знайдено: {best_model}")


## 10. Тестування на прикладі зображення (опціонально)


In [ ]:
# Знаходимо тестове зображення
test_images_dir = os.path.join(YOLO_DATASET_DIR, 'test', 'images')
if os.path.exists(test_images_dir):
    test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if test_images:
        test_image_path = os.path.join(test_images_dir, test_images[0])
        print(f"Тестування на зображенні: {test_image_path}")
        
        if os.path.exists(best_model):
            test_model = YOLO(best_model)
            
            # Передбачення
            predictions = test_model.predict(
                source=test_image_path,
                conf=0.25,
                save=True,
                save_dir=os.path.join(OUTPUT_DIR, 'predictions'),
                show_labels=True,
                show_boxes=True
            )
            
            # Показуємо результат
            for result in predictions:
                print(f"\nЗнайдено об'єктів: {len(result.boxes)}")
                if len(result.boxes) > 0:
                    print("\nДеталі детекцій:")
                    for i, box in enumerate(result.boxes):
                        cls = int(box.cls[0])
                        conf = float(box.conf[0])
                        class_name = test_model.names[cls]
                        print(f"   {i+1}. {class_name}: {conf:.2%}")
                    
                    # Показуємо зображення з детекціями
                    from IPython.display import Image, display
                    result_path = os.path.join(OUTPUT_DIR, 'predictions', test_images[0])
                    if os.path.exists(result_path):
                        display(Image(result_path))
        else:
            print(f"ПОМИЛКА: Модель не знайдено: {best_model}")
    else:
        print(f"ПОМИЛКА: Тестові зображення не знайдено в {test_images_dir}")
else:
    print(f"ПОМИЛКА: Тестова директорія не знайдено: {test_images_dir}")


## Примітки

- Для кращих результатів рекомендується використовувати GPU
- Розмір батчу залежить від доступної пам'яті GPU
- Моделі більшого розміру (l, x) потребують більше пам'яті та часу навчання
- Рекомендується почати з моделі `n` (nano) для швидкого тестування
- Змініть параметри `MODEL_SIZE`, `EPOCHS`, `BATCH_SIZE` у комірці 7 для налаштування навчання
